In [19]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# load the IMU data into a numpy array
imu_data = np.genfromtxt('CSV3.csv', delimiter=',')

# convert the data to PyTorch tensors
imu_data = torch.from_numpy(imu_data)
X = imu_data[:, 4:10]
y = imu_data[:, 2:4]
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# define the model architecture
class IMUModel(torch.nn.Module):
    def __init__(self):
        super(IMUModel, self).__init__()
        self.lstm = torch.nn.LSTM(6, 32)  # 6 is the number of dimensions in the IMU data
        self.linear1 = torch.nn.Linear(32, 6)
        self.linear2 = torch.nn.Linear(32, 6)

    def forward(self, x):
        x, _ = self.lstm(x)
        return self.linear1(x), self.linear2(x)

# create the model and move it to the GPU (if available)
model = IMUModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# define the contrastive loss function
def contrastive_loss(y_true, y_pred):
  margin = 1
  max_val, _ = torch.max(margin - y_pred, 0)
  return torch.mean(y_true * torch.square(y_pred) + (1 - y_true) * torch.square(max_val))

# compile the model with a contrastive loss function
optimizer = torch.optim.Adam(model.parameters())

# train the model on the training data
for epoch in range(10):  # train for 10 epochs
    # Forward pass
    x = X_train.to(torch.float)

    output1, output2 = model(x)

    # Compute the contrastive loss
    loss = contrastive_loss(output1, output2)
    print(loss)
    # Backpropagate and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor(1.7755, grad_fn=<MeanBackward0>)
tensor(1.7106, grad_fn=<MeanBackward0>)
tensor(1.6565, grad_fn=<MeanBackward0>)
tensor(1.6091, grad_fn=<MeanBackward0>)
tensor(1.5673, grad_fn=<MeanBackward0>)
tensor(1.5284, grad_fn=<MeanBackward0>)
tensor(1.4903, grad_fn=<MeanBackward0>)
tensor(1.4508, grad_fn=<MeanBackward0>)
tensor(1.4187, grad_fn=<MeanBackward0>)
tensor(1.3858, grad_fn=<MeanBackward0>)
